In [42]:
#读取dnd文件
f=open(r"C:\DNDC\RW250.dnd")
content = f.read()
print(content)

DNDC_Input_Parameters
----------------------------------------
Site_infomation
                        
__Site_name                                                       New Site Name
__Simulated_years                                                     1
__Latitude                                                      36.1700
__Daily_record                                                        1
__Unit_system                                                         0
__None                                                                0
__None                                                                0
__None                                                                0
__None                                                                0
__None                                                                0
----------------------------------------
Climate_data                            

__Climate_data_type                                                   0
__N_in_rainfa

In [1]:
#对参数进行蒙特卡洛模拟(uniform)
import re
import random

# 读取DND文件内容
with open("C:/DNDC/data/.dnd", 'r') as file:
    dnd_data = file.read()

# 定义要替换的参数和变化范围
parameters = {
    "______Manure_amount": 0.2,  # 参数的变化范围为20%
    "______Manure_C/N": 0.2, 
    "__Bulk_density": 0.2,  
    "__pH": 0.2,
    "______Grain_C/N": 0.2,
    "__Top_layer_SOC": 0.2,
    "__Clay_fraction": 0.2,
    "______Urea": 0.2            
}

# 定义循环次数
num_iterations = 5000

# 使用正则表达式匹配参数
pattern = r'(' + '|'.join(map(re.escape, parameters.keys())) + r')(\s+)(\S+)'
matches = re.findall(pattern, dnd_data)

# 进行循环，并生成多个修改后的文件
for i in range(num_iterations):
    modified_dnd_data = dnd_data
    for match in matches:
        parameter = match[0]
        old_value = match[2]
        change_range = parameters[parameter]
        new_value = str(float(old_value) * (1 + random.uniform(-change_range, change_range)))
        modified_dnd_data = modified_dnd_data.replace(match[0] + match[1] + old_value, match[0] + match[1] + new_value, 1)

    # 构建新的文件名
    new_file_name = f"C:/DNDC/data/RW400_modified_{i+1}.dnd"

    # 将修改后的内容写入新的文件
    with open(new_file_name, 'w') as file:
        file.write(modified_dnd_data)

    print(f"output：{new_file_name}")

print("successfully run!")


FileNotFoundError: [Errno 2] No such file or directory: 'C:/DNDC/dnd'

In [11]:
import os

# 定义要读取的文件夹和输出文件
folder_path = "C:/DNDC/data/"
output_file = "C:/DNDC/batch_list.txt"

# 获取文件夹内所有文件路径
file_paths = []
for filename in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, filename)):
        file_paths.append(os.path.join(folder_path, filename))

# 写入文件名到输出文件中
with open(output_file, "w", encoding="utf-8") as f:
    # 第一行写入文件总数
    f.write(f"{len(file_paths)}\n")
    # 逐行写入文件名
    for path in file_paths:
        f.write(f"{path}\n")
    print(f"save '{output_file}' ")

save 'C:/DNDC/batch_list.txt' 


In [44]:
import re
import pandas as pd
import os

# 定义要查找的参数列表
parameters = [
    "______Manure_amount",
    "______Manure_C/N",
    "__Bulk_density",
    "__pH",
    "______Grain_C/N",
    "__Top_layer_SOC",
    "__Clay_fraction",
    "______Urea"
]

# 定义文件名的前缀和循环次数
file_prefix = r"C:\DNDC\data\RW400_modified_"  # 文件名前缀
num_iterations = 5000  # 循环次数

# 创建一个空字典用于存储参数名和每次值的列表
parameter_values = {}

# 循环读取文件并提取参数值
for i in range(num_iterations):
    file_name = file_prefix + str(i+1) + '.dnd'

    # 读取文件内容
    with open(file_name, 'r') as file:
        dnd_data = file.readlines()

    # 初始化参数值列表
    param_values = {}

    # 逐行匹配参数值
    for line in dnd_data:
        for parameter in parameters:
            pattern = re.escape(parameter) + r'(\s+)(\S+)'
            matches = re.findall(pattern, line)
            if matches:
                if parameter not in param_values:
                    param_values[parameter] = []
                for match in matches:
                    var_name = f"{parameter}_{len(param_values[parameter])+1}"
                    param_values[parameter].append((var_name, match[1]))
    
    # 保存参数值
    for parameter, values in param_values.items():
        for var_name, value in values:
            if var_name not in parameter_values:
                parameter_values[var_name] = []
            parameter_values[var_name].append(value)

# 生成表格
df = pd.DataFrame(parameter_values)

# 给第一列加上循环次数
df.insert(0, "number", range(1, num_iterations+1))

# 保存到桌面
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
save_path = os.path.join(desktop_path, "parameter_values.csv")
df.to_csv(save_path, index=False)

print(f"save: {save_path}")


表格已保存到桌面: C:\Users\mz253\Desktop\parameter_values.csv
